<a href="https://colab.research.google.com/github/armahdavi/nlp_document_tracking_construction_management/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file = '/content/sentiment_list - Label.xlsx'

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Read the Excel file
df = pd.read_excel(file)

# Clean and prepare the data
df = df[['List', 'Maj. Def']]
df.dropna(inplace=True)
df = df[df['Maj. Def'] != 'Remove']

# Rename columns to match the preprocessing function
df.columns = ['text', 'label']

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
def preprocess_dataframe_tf(df, \='bert-base-uncased', max_length=128):
    """Preprocesses a pandas DataFrame for BERT using TensorFlow."""
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    texts = df['text'].tolist()
    labels = tf.convert_to_tensor(df['label'].tolist())

    encoded_inputs = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    return encoded_inputs['input_ids'], encoded_inputs['attention_mask'], labels

def create_bert_model(bert_model_name='bert-base-uncased', num_labels=2):
    """Creates a BERT model for classification using TensorFlow."""
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

    bert_model = TFBertModel.from_pretrained(bert_model_name)
    outputs = bert_model(input_ids, attention_mask=attention_mask)

    pooled_output = outputs.pooler_output
    dropout = tf.keras.layers.Dropout(0.1)(pooled_output)
    output = tf.keras.layers.Dense(num_labels, activation='softmax')(dropout)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    return model

In [ ]:
# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Preprocess train and test data
train_input_ids, train_attention_masks, train_labels = preprocess_dataframe_tf(train_df)
test_input_ids, test_attention_masks, test_labels = preprocess_dataframe_tf(test_df)

# Determine number of unique labels
num_labels = len(df['label'].unique())

# Create the BERT model
model = create_bert_model(num_labels=num_labels)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(
    [train_input_ids, train_attention_masks],
    train_labels,
    epochs=3,
    batch_size=16,
    validation_split=0.2
)

# Predict and evaluate
train_predictions = model.predict([train_input_ids, train_attention_masks])
test_predictions = model.predict([test_input_ids, test_attention_masks])

train_predicted_labels = tf.argmax(train_predictions, axis=1).numpy()
test_predicted_labels = tf.argmax(test_predictions, axis=1).numpy()

train_accuracy = accuracy_score(train_labels.numpy(), train_predicted_labels)
test_accuracy = accuracy_score(test_labels.numpy(), test_predicted_labels)

train_recall = recall_score(train_labels.numpy(), train_predicted_labels, average='weighted')
test_recall = recall_score(test_labels.numpy(), test_predicted_labels, average='weighted')

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Train Recall: {train_recall}")
print(f"Test Recall: {test_recall}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
40/40 [==============================] - 70s 528ms/step - loss: 0.5766 - accuracy: 0.7962 - val_loss: 0.3193 - val_accuracy: 0.9187
Epoch 2/3
40/40 [==============================] - 17s 416ms/step - loss: 0.3261 - accuracy: 0.8762 - val_loss: 0.3197 - val_accuracy: 0.9312
Epoch 3/3
7/7 [==============================] - 2s 228ms/step
Train Accuracy: 0.9611528822055138
Test Accuracy: 0.925
Train Recall: 0.9611528822055138
Test Recall: 0.925


In [ ]:
train_input_ids, train_attention_masks, train_labels

(<tf.Tensor: shape=(798, 128), dtype=int32, numpy=
 array([[  101,  4654,  2361, ...,     0,     0,     0],
        [  101, 21086,  2000, ...,     0,     0,     0],
        [  101, 21086,  2012, ...,     0,     0,     0],
        ...,
        [  101,  1996, 21086, ...,     0,     0,     0],
        [  101, 21086,  2000, ...,     0,     0,     0],
        [  101,  2035,  3653, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(798, 128), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(798,), dtype=int32, numpy=
 array([3, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2,
        3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 3, 2, 3,
        3, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1, 2, 2, 2, 3, 2, 2, 1, 2,
        3, 1, 2, 2, 3, 2, 2, 2,

In [ ]:
# Keep unlabeled data for predictions
df_unlabeled = pd.read_excel(file)
df_unlabeled = df_unlabeled[['List', 'Maj. Def']]
df_unlabeled = df_unlabeled[df_unlabeled['Maj. Def'].isna()]
df_unlabeled.columns = ['text', 'label']

,text,label
710,At the time of the site visit EXP observed tha...,NaN
711,At the time of the site visit EXP observed tha...,NaN
712,At the time of the site visit EXP observed tha...,NaN
713,At the time of the site visit window wall inst...,NaN
714,At the time of the site visit window wall inst...,NaN
...,...,...
2932,Windows installed on east elevation weren’t ac...,NaN
2933,"Windows panel # 1, behind column was installed...",NaN
2934,"Windows panel # 1-3, behind column was install...",NaN
2935,Windows panel # 40 installed behind a column o...,NaN


In [ ]:
unlabeled_input_ids, unlabeled_attention_masks, _ = preprocess_dataframe_tf(df_unlabeled)
unlabeled_predictions = model.predict([unlabeled_input_ids, unlabeled_attention_masks])
unlabeled_predictions_labels = tf.argmax(unlabeled_predictions, axis=1).numpy()
unlabeled_predictions_labels

61/61 [==============================] - 16s 264ms/step


array([2, 2, 2, ..., 3, 3, 2])

In [ ]:
label_mapping = {0: 'FCNBO', 1: 'Fixed', 2: 'N', 3: 'Y'}
string_labels = [label_mapping[label] for label in unlabeled_predictions_labels]


In [ ]:
results_df = pd.DataFrame({'text': df_unlabeled['text'], 'predicted_label': string_labels})
results_df.to_excel("text_and_predicted_labels.xlsx", index=False)